#Rede neural com Sklearn utilizando o dataset do Titanic

In [ ]:
#importando as bibliotecas necessárias
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import SelectKBest, SelectPercentile, f_regression, f_classif
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
#carregando o datset - disponível em https://www.kaggle.com/competitions/titanic
TitanicDF  = pd.read_csv('Titanic-Dataset.csv', encoding='latin-1', sep=',')
TitanicDF.info()
TitanicDF.describe()

In [ ]:
#Avaliando o domínio dos dados nulos
print('-------- Age ----------')
print(f"Distinct entries: {len(TitanicDF.groupby(['Age']))}")
print(TitanicDF['Age'].unique)
print('-------- Cabin ----------')
print(f"Distinct entries: {len(TitanicDF.groupby(['Cabin']))}")
print(TitanicDF['Cabin'].unique)
print('-------- Embarked ----------')
print(f"Distinct entries: {len(TitanicDF.groupby(['Embarked']))}")
print(TitanicDF['Embarked'].unique)

In [ ]:
#preenchendo campos nulos
mediaIdade = TitanicDF['Age'].mean()
TitanicDF.update(TitanicDF['Age'].fillna(mediaIdade))
TitanicDF.update(TitanicDF['Cabin'].fillna('ABC'))
TitanicDF.update(TitanicDF['Embarked'].fillna('S'))

In [ ]:
#analisando o dataset
TitanicDF.info()
TitanicDF.head()

In [ ]:
#### plotando a correlação das variaveis
plt.figure(figsize=(10, 7))
sns.heatmap(TitanicDF.corr(),
            annot = True,
            fmt = '.2f',
            cmap='Blues')
plt.title('Correlação entre variáveis')
plt.show()

In [ ]:
#separando a base em treino e validação
DFTreino, DFValidacao = train_test_split(TitanicDF, test_size=0.20, random_state=42)

#Avaliando as bases
print(f"Base de Treino...: {DFTreino.shape}")
print(f"Base de Validação: {DFValidacao.shape}")

Base de Treino...: (712, 12)
Base de Validação: (179, 12)


In [ ]:
#normalizando os campos categoricos
lbE = LabelEncoder()
DFTreino['Sex'] = lbE.fit_transform(DFTreino['Sex'])
DFTreino['Embarked'] = lbE.fit_transform(DFTreino['Embarked'])
DFTreino['Ticket'] = lbE.fit_transform(DFTreino['Ticket'])
DFTreino['Cabin'] = lbE.fit_transform(DFTreino['Cabin'])

#remover campos que não fazem sentido
X = DFTreino.drop(["Name","Survived","PassengerId"], axis=1)
y = DFTreino["Survived"]

In [ ]:
#Quebrando a base de dados de TREINO para teste e validação
X_treino, X_teste, y_treino, y_teste = train_test_split(X,
                                                        y,
                                                        test_size = 0.30,
                                                        random_state = 40)
#Avaliando as bases
X_treino.info()
y_treino.info()

In [ ]:
#Verificar a importância de cada variável através do KBest
k = 7
melhores = SelectKBest(score_func=f_regression, k=k)
pontuacao = melhores.fit(X_treino,y_treino)
features  = pontuacao.transform(X_treino)
#Mostra resultados

print('\nQtde Original de variáveis: ', X_treino.shape[1])
print('\nQtde depois da redução   :', features.shape[1])

#Mostrando as colunas selecionadas
mask = melhores.get_support()
print('Colunas selecionadas: ',X_treino.columns[mask])

#grafico das melhores
pd.Series(melhores.scores_, index=X_treino.columns).sort_values().plot.barh()

In [ ]:
#deixando apenas aas caracteristicas escolhida
selecao = ['Pclass', 'Sex', 'Age', 'Ticket', 'Fare', 'Cabin', 'Embarked']

X_treino = X_treino[selecao]
X_teste  = X_teste[selecao]

print(X_treino.shape)
print(X_teste.shape)
print(X_treino.info())

In [ ]:
mlp = MLPClassifier(hidden_layer_sizes=(200,), activation='relu', solver='adam', max_iter=800)
mlp.fit(X_treino,y_treino)

predict_treino = mlp.predict(X_treino)
predict_teste  = mlp.predict(X_teste)

In [ ]:
#matriz de confusão
print('Matriz de Confusão')
print(confusion_matrix(y_treino,predict_treino))
print('***-----------------***')

print('Análise predição')
print(classification_report(y_treino,predict_treino))